In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

/home/roy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = np.loadtxt('./ML_Data/CLIENT04_ML.csv', delimiter=',')


In [3]:
samples=2000000
X = data[:samples,1:-18]


In [4]:
Y = data[:samples,-2:]
PC_Address=data[:samples,0]

In [5]:
#print(PC_Address.shape,Y.shape)
#x=int(PC_Address[1000]%10000)
#print(x)
#print(PC_Address[x])


In [6]:
n_nodes_hl1 = 100
n_nodes_hl2 = 100
n_nodes_hl3 = 100
n_classes = 2
#Global Histroy with size of register 100
GHR=np.zeros((1,100))
#Local History with 1000 enteries of Local History parsed with the address. Each register size if 10
LHR=np.zeros((1000,10))
result=np.zeros((1,samples))


x = tf.placeholder('float', [None, 126])
y = tf.placeholder('float',[None,2])
def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([126, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output


def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    train_error= tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    hm_epochs = 1
    temp=0
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            epoch_loss = 0
            acc=0
            for i in range(int(samples)):
                #local history table address index found using the address
                add1=int(PC_Address[i]%1000)
                # the input vector to the Neural network that has GHR, LHR and PC
                epoch_x=np.concatenate((X[i:i+1],GHR,LHR[add1:add1+1]),axis=1)
                # the actual output
                epoch_y=Y[i:i+1]
                epoch_y.shape=(1,2)
                # check if the output is correct and store in eq
                eq=sess.run([train_error],feed_dict={x:epoch_x, y:epoch_y})
                # run the optimizer update the weigths
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                # increase the hits if the prediction is correct
                acc+=eq[0]
                epoch_loss += c
                #updata GHR
                GHR[0,:-1]=GHR[0,1:]
                GHR[0,-1]=Y[i:i+1,0]
                #to store the result
                result[0,i]=acc
                #update LHR
                LHR[add1,:-1]=LHR[add1,1:]
                LHR[add1,-1]=Y[i:i+1,0]
                


train_neural_network(x)

Epoch 0 completed out of 1 loss: 1839233.81822
Accuracy:  [ 0.92356996]
Accuracy:  [1847139]
[[  1.00000000e+00   2.00000000e+00   3.00000000e+00 ...,   1.84713700e+06
    1.84713800e+06   1.84713900e+06]]


In [7]:
# print the total accuracy after every 100 thousand branches.
temp=0
print(len(result[0]))
for i in range(int(len(result[0])/10000)):
    print(i,"cycle accuracy",(result[0,i*10000]-temp)/10000)
    temp=result[0,i*10000]

2000000
0 cycle accuracy 0.0001
1 cycle accuracy 0.7247
2 cycle accuracy 0.8039
3 cycle accuracy 0.7738
4 cycle accuracy 0.8247
5 cycle accuracy 0.7704
6 cycle accuracy 0.8175
7 cycle accuracy 0.8012
8 cycle accuracy 0.8093
9 cycle accuracy 0.7906
10 cycle accuracy 0.7957
11 cycle accuracy 0.8211
12 cycle accuracy 0.8196
13 cycle accuracy 0.8427
14 cycle accuracy 0.8637
15 cycle accuracy 0.8737
16 cycle accuracy 0.8356
17 cycle accuracy 0.8391
18 cycle accuracy 0.8683
19 cycle accuracy 0.881
20 cycle accuracy 0.8299
21 cycle accuracy 0.8453
22 cycle accuracy 0.865
23 cycle accuracy 0.8711
24 cycle accuracy 0.8535
25 cycle accuracy 0.8633
26 cycle accuracy 0.8568
27 cycle accuracy 0.9072
28 cycle accuracy 0.8747
29 cycle accuracy 0.8725
30 cycle accuracy 0.8841
31 cycle accuracy 0.8968
32 cycle accuracy 0.8916
33 cycle accuracy 0.9095
34 cycle accuracy 0.9233
35 cycle accuracy 0.9002
36 cycle accuracy 0.9737
37 cycle accuracy 0.9579
38 cycle accuracy 0.9712
39 cycle accuracy 0.8911
40 c